In [1]:
#load all necessary packages
import numpy as np
import pandas as pd
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from scipy.interpolate import interp1d

import warnings
warnings.filterwarnings('ignore')

import tensorflow.compat.v1 as tf
import tensorflow_probability as tfp
import edward2 as ed
tfd = tfp.distributions
tf.disable_eager_execution()

sns.set()
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
%config InlineBackend.figure_format = 'svg'
np.random.seed(111)
tf.set_random_seed(111)

C:\Users\Ethan\AppData\Roaming\Python\Python39\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
train = pd.read_csv('../TrainTestData/com_train.csv')
test = pd.read_csv('../TrainTestData/com_test.csv')

In [4]:
#split, normalize the train-test data
def preprocess(train,test):    
    x = train.iloc[:,:35]
    y = train.iloc[:,35]
    x_val = test.iloc[:,:35]
    y_val = test.iloc[:,35]
    
    temp = x.astype('float32')
    x = temp.copy()
    
    temp_val = x_val.astype('float32')
    x_val = temp_val.copy()
    
    y = y.values
    y = y.reshape(-1,1)
    y_val = y_val.values
    y_val = y_val.reshape(-1,1)
    y_unnormed = y_val.copy()
    
    return x,y,x_val,y_val,y_unnormed

x,y,x_val,y_val,y_unnormed = preprocess(train,test)

In [5]:
D = x.shape[1]
N = x.shape[0]
noise_std_true = 1.0
#define y ~ N(coeffs * x + bias , noise)
def linear_regression(features):
    D = features.shape[1]  
    # coefficients: (D,1)
    coeffs = ed.Normal(loc=tf.zeros([D,1]),scale=tf.ones([D,1]),name="coeffs")
    #bias: (1)
    bias = ed.Normal(loc=tf.zeros([1]), scale=tf.ones([1]),name="bias") 
    #noise: (1)
    noise_std = ed.HalfNormal(scale=tf.ones([1]),name="noise_std")
    # prediction: y ~ N(coeffs * x + bias , noise)
    predictions = ed.Normal(loc=tf.matmul(features, coeffs)+bias,scale=noise_std,name="predictions")
    return predictions

In [6]:
#create log joint probability function 
log_joint = ed.make_log_joint_fn(linear_regression)
def target_log_prob_fn(coeffs, bias, noise_std):
    return log_joint(
        features=x, 
        coeffs=coeffs, bias=bias, noise_std=noise_std, 
        predictions=y
    )

In [7]:
#number of total iterations
num_results = int(5e3) 
#number of burn-in steps
n_burnin = int(2.5e3)
#stepsize for leapfrog method
step_size = 0.01
# Parameter sizes
coeffs_size = [D,1]
bias_size = [1]
noise_std_size = [1]

#define the NUTS sampler as kernel
kernel = tfp.mcmc.NoUTurnSampler(
    target_log_prob_fn=target_log_prob_fn,
    step_size=step_size)

#create the sample chain
states, kernel_results = tfp.mcmc.sample_chain(
    num_results=num_results,
    num_burnin_steps=n_burnin,
    kernel=kernel,
    current_state=[
        tf.zeros(coeffs_size, name='init_coeffs'),
        tf.zeros(bias_size, name='init_bias'),
        tf.ones(noise_std_size, name='init_noise_std'),
    ])
coeffs, bias, noise_std = states

In [8]:
class Timer:
    def __enter__(self):
        self.t0 = time.time()
    def __exit__(self, *args):
        print('Elapsed time: %0.2fs' % (time.time()-self.t0))
        
#run the chain
with Timer(), tf.Session() as sess:
  [
      coeffs_,
      bias_,
      noise_std_,
      is_accepted_,
  ] = sess.run([
      coeffs,
      bias,
      noise_std,
      kernel_results.is_accepted,
  ])

# Samples after burn-in
coeffs_samples = coeffs_[n_burnin:,:,0]
bias_samples = bias_[n_burnin:]
noise_std_samples = noise_std_[n_burnin:]
accepted_samples = is_accepted_[n_burnin:]

Elapsed time: 279.45s


In [10]:
accepted_samples

array([False, False, False, ..., False, False, False])

In [9]:
#check for acceptance rate
print('Acceptance rate: %0.1f%%' % (100*np.mean(accepted_samples)))

Acceptance rate: 0.0%


In [22]:
def ind_pred_dist(X):
    '''Compute the prediction distribution'''
    predictions = (np.matmul(X, coeffs_samples.transpose()) + 
                 bias_samples[:,0])
    noise = (noise_std_samples[:,0] * 
           np.random.randn(noise_std_samples.shape[0]))
    return predictions + noise

# Compute prediction distribution for all validation samples
N_val = y_val.shape[0]
Nmcmc = coeffs_samples.shape[0]
prediction_distribution = np.zeros((N_val, Nmcmc))
for i in range(N_val):
    prediction_distribution[i,:] = ind_pred_dist(x_val[i,:])

# Plot random datapoints and their prediction intervals
fig, axes = plt.subplots(4, 2, sharex='all')
for i in range(4):
    for j in range(2):
        ix = i*2+j
        pred_dist = prediction_distribution[ix,:]
        sns.kdeplot(pred_dist, fill=True, ax=axes[i][j])
        axes[i][j].axvline(x=y_val[ix,0])
    axes[i][0].set_ylabel('p(y)')

axes[3][0].set_xlabel('y')
axes[3][1].set_xlabel('y')

InvalidIndexError: (0, slice(None, None, None))

In [21]:
pred_dist = prediction_distribution[100,:]

fig, ax = plt.subplots(figsize=(5, 4))
sns.kdeplot(pred_dist, fill=True)

density, x = sns.kdeplot(pred_dist, fill=False).get_lines()[0].get_data()
f = interp1d(density, x)
y = f(y_val[100,0])
print('Probability of wine quality as {} is {:0.2f}%'.format(y_unnormed[100,0],y*100))
ax.axvline(x=y_val[100,0])
ax.axvline(np.quantile(pred_dist, 0.025), color='r', linestyle="dashed")
ax.axvline(np.quantile(pred_dist, 0.975), color='r', linestyle="dashed")

plt.show()

NameError: name 'prediction_distribution' is not defined